In [14]:
import pandas as pd
import numpy as np

**read_data_small** is the function to read in the small dataset about 30 MB

In [15]:
def read_data_small():
    X_train_full = pd.read_csv("data_small/X_train_small.csv")
    X_test = pd.read_csv("data_small/X_test_small.csv")
    y_train_full = np.asarray(pd.read_csv("data_small/y_train_small.csv", header=None)[0])
    
    X_train = X_train_full[0: 99275]
    X_valid = X_train_full[99275:]
    
    y_train = y_train_full[0:99275]
    y_valid = y_train_full[99275:]
    return X_train, X_valid, X_test, y_train, y_valid

**read_data_big** is the function to read in the big dataset about 100 MB

In [16]:
def read_data_big():
    X_train = pd.read_csv("data_big/X_train_big.csv")
    X_test = pd.read_csv("data_big/X_test_big.csv")
    y_train = np.asarray(pd.read_csv("data_big/y_train_big.csv", header=None)[0])
    return X_train, X_test, y_train

**read_data** is the function to read in the whole dataset about 1.5 G

In [17]:
def read_data():
    X_train = pd.read_csv("data/X_train.csv")
    X_test = pd.read_csv("data/X_test.csv")
    y_train = np.asarray(pd.read_csv("data/y_train.csv", header=None)[0])
    return X_train, X_test, y_train

In [18]:
X_train, X_valid, X_test, y_train, y_valid = read_data_small()

In [19]:
np.shape(X_test)

(60782, 29)

In [20]:
np.shape(X_train)

(99275, 29)

# Insert Your Code Here

**detect_spoofying** is the function for training the classifier and classify the results. 

Here we provide an simple example.

In [89]:
### import libraries here ###
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_validate

### code classifier here ###
def format_data(df):
    
    # append numberical columns
    rst = df.loc[:,["price","volume","bestBid","bestAsk",'bestBidVolume',
                    'bestAskVolume','lv2Bid', 'lv2BidVolume','lv2Ask', 
                    'lv2AskVolume', 'lv3Bid', 'lv3BidVolume', 'lv3Ask',
                    'lv3AskVolume']]
    
    # encode the binaries
    rst["isBid"] = df.isBid*1
    rst["isBuyer"] = df.isBuyer*1
    rst["isAggressor"] = df.isAggressor*1
    rst["type"] = (df.type == "ORDER")*1
    rst["source"] = (df.source=="USER")*1
    
    # parse the order id data
    rst["orderId"] = df.orderId.str.split('-').str[-1]
    rst["tradeId"] = df.tradeId.str.split('-').str[-1]
    rst["bidOrderId"] = df.bidOrderId.str.split('-').str[-1]
    rst["askOrderId"] = df.askOrderId.str.split('-').str[-1]
    
    # encode the multiple lable data
    # df.operation.unique()[:-1] gets you every row except nan. So, this creates a dataframe out of each unique operation except nan
    tmp_operation = pd.DataFrame(pd.get_dummies(df.operation), columns=df.operation.unique()[:-1])
    rst = pd.concat([rst, tmp_operation], axis=1)
    # gets you a one-hot encoding 
    tmp_endUserRef = pd.DataFrame(pd.get_dummies(df.endUserRef), columns=df.endUserRef.unique()[:-1])
    rst = pd.concat([rst, tmp_endUserRef], axis=1)
    
    # also feel free to add more columns inferred from data
    # smartly engineered features can be very useful to improve the classification resutls
    rst["timeSinceLastTrade"] = X_train[["timestamp","endUserRef"]].groupby("endUserRef").diff()
    
    return rst


def data_prep(X_train, X_test, y_train, X_valid, y_valid):
    # clean up the data
    X_clean = format_data(pd.concat([X_train, X_valid, X_test]))
    X_clean = X_clean.fillna(-1)
    
    X_train_clean = X_clean.iloc[:X_train.shape[0],:]
    X_valid_clean = X_clean.iloc[X_train.shape[0]:(X_train.shape[0]+X_valid.shape[0]),:]
    X_test_clean = X_clean.iloc[(X_train.shape[0]+X_valid.shape[0]):,:]
    
    X_train_clean_scaled = scale(X_train_clean)
    X_valid_clean_scaled = scale(X_valid_clean)
    X_test_clean_scaled = scale(X_test_clean)
    
    print("shapes after data prep")
    print(X_train_clean_scaled.shape)
    print(X_valid_clean_scaled.shape)
    print(X_test_clean_scaled.shape)
    
    
#     X_valid_clean = format_data(X_valid)
#     X_valid_clean = X_valid_clean.fillna(-1)
    
    
    return X_train_clean_scaled, X_test_clean_scaled, X_valid_clean_scaled, y_train, y_valid

In [24]:
X_train_clean_scaled, X_test_clean_scaled, x_valid_clean_scaled, y_train, y_valid = data_prep(X_train, X_test, y_train, X_valid, y_valid)
# X_valid_clean_scaled, X_test_clean_scaled, y_valid = data_prep(X_valid, X_test, y_valid)

(202604, 497)
(99275, 497)
(42547, 497)
(60782, 497)
(99275, 497)
(42547, 497)
(60782, 497)


In [93]:
def detect_spoofying_logistic(X_train_clean_scaled, x_valid_clean_scaled, X_test_clean_scaled, y_train, y_valid):
    
    # fit classifier
    clf = LogisticRegression(random_state=0, class_weight='balanced').fit(X_train_clean_scaled, y_train)
    y_train_prob_pred = clf.predict_proba(X_train_clean_scaled)
    y_valid_prob_pred = clf.predict_proba(x_valid_clean_scaled)
    y_test_prob_pred = clf.predict_proba(X_test_clean_scaled)
    
    return y_train_prob_pred, y_valid_prob_pred, y_test_prob_pred

In [94]:
clf = LogisticRegression(random_state=0, class_weight='balanced').fit(X_train_clean_scaled, y_train)

D:\Users\tehtea\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Users\tehtea\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [95]:
y_train_prob_pred = clf.predict_proba(X_train_clean_scaled)

In [96]:
y_valid_prob_pred = clf.predict_proba(x_valid_clean_scaled)

In [97]:
y_test_prob_pred = clf.predict_proba(X_test_clean_scaled)

**score** is the function that we use to compare the results. An example is provided with scoring the predictions for the training dataset. True labels for the testing data set will be supplied to score the predictions for testing dataset.

Score is based on cohen's kappa measurement. https://en.wikipedia.org/wiki/Cohen%27s_kappa

In [98]:
from sklearn.metrics import cohen_kappa_score

def score(y_pred, y_true):
    """
    y_pred: a numpy 4d array of probabilities of point assigned to each label
    y_true: a numpy array of true labels
    """
    y_pred_label = np.argmax(y_pred, axis=1)
    return cohen_kappa_score(y_pred_label, y_true)

**wrapper** is the main function to read in unzipped data and output a score for evaluation. In addition, the function returns the y probability matrix (both train and test) for grading. More details about submitting format are outlined below.

In [99]:
def wrapper(spoof_detector):
    # read in data
    X_train, X_valid, X_test, y_train, y_valid = read_data_small()
    # or if you have the computational power to work with the big data set, 
    # you can comment out the read_data_samll line and uncomment the following read_data_big
    # X_train, X_test, y_train = read_data_big()
    
    # process the data, train classifier and output probability matrix
#     X_train_clean_scaled, X_test_clean_scaled, y_train = data_prep(X_train, X_test, y_train)
    X_train_clean_scaled, X_test_clean_scaled, x_valid_clean_scaled, y_train, y_valid = data_prep(X_train, X_test, y_train, X_valid, y_valid)

    y_train_prob_pred, y_valid_prob_pred, y_test_prob_pred = spoof_detector(X_train_clean_scaled, x_valid_clean_scaled, X_test_clean_scaled, y_train, y_valid)
    
    # score the predictions
    print("scoring the results")
    print(y_train_prob_pred.shape)
    print(y_train.shape)
    score_train = score(y_train_prob_pred, y_train)
    print(y_valid_prob_pred.shape)
    print(y_valid.shape)
    score_valid = score(y_valid_prob_pred, y_valid)
    # score_test = score(y_test_prob_pred, y_test)
    
    # return the scores
    return score_train, score_valid, y_train_prob_pred, y_valid_prob_pred, y_test_prob_pred

Call function wrapper:

In [100]:
score_train, score_valid, y_train_prob_pred, y_valid_prob_pred, y_test_prob_pred = wrapper(detect_spoofying_logistic)

shapes after data prep
(99275, 497)
(42547, 497)
(60782, 497)


D:\Users\tehtea\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Users\tehtea\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


scoring the results
(99275, 3)
(99275,)
(42547, 3)
(42547,)


Score for training data set is:

In [101]:
score_train

0.6530360000368849

In [102]:
score_valid

0.35920167794858204

In [103]:
from sklearn.ensemble import RandomForestClassifier

# def detect_spoofying_rf(X_train, X_test, y_train):
def detect_spoofying_rf(X_train_clean_scaled, x_valid_clean_scaled, X_test_clean_scaled, y_train, y_valid):
    
    # fit classifier
    clf = RandomForestClassifier(max_depth=100, random_state=0, class_weight='balanced').fit(X_train_clean_scaled, y_train)
    y_train_prob_pred = clf.predict_proba(X_train_clean_scaled)
    y_valid_prob_pred = clf.predict_proba(x_valid_clean_scaled)
    y_test_prob_pred = clf.predict_proba(X_test_clean_scaled)
    
    return y_train_prob_pred, y_valid_prob_pred, y_test_prob_pred

In [104]:
score_train, score_valid, y_train_prob_pred, y_valid_prob_pred, y_test_prob_pred = wrapper(detect_spoofying_rf)

shapes after data prep
(99275, 497)
(42547, 497)
(60782, 497)


D:\Users\tehtea\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


scoring the results
(99275, 3)
(99275,)
(42547, 3)
(42547,)


In [105]:
score_train

0.9720251390734232

In [106]:
score_valid

0.005325034727520661

In [109]:
from sklearn.ensemble import GradientBoostingClassifier

# def detect_spoofying_gb(X_train, X_test, y_train):
def detect_spoofying_gb(X_train_clean_scaled, x_valid_clean_scaled, X_test_clean_scaled, y_train, y_valid):
    
#     # clean up the data
#     X_clean = format_data(pd.concat([X_train, X_test]))
#     X_clean = X_clean.fillna(-1)
#     X_train_clean = X_clean.iloc[:X_train.shape[0],:]
#     X_test_clean = X_clean.iloc[X_train.shape[0]:,:]
#     X_train_clean_scaled = scale(X_train_clean)
#     X_test_clean_scaled = scale(X_test_clean)

    original_params = {'n_estimators': 1000, 'max_leaf_nodes': 4, 'max_depth': None, 'random_state': 2,
                   'min_samples_split': 5}
    params = dict(original_params)
    # fit classifier
    clf = GradientBoostingClassifier(**params).fit(X_train_clean_scaled, y_train)
    y_train_prob_pred = clf.predict_proba(X_train_clean_scaled)
    y_test_prob_pred = clf.predict_proba(X_test_clean_scaled)
    
    return y_train_prob_pred, y_test_prob_pred

In [ ]:
score_train, y_train_prob_pred, y_test_prob_pred = wrapper(detect_spoofying_gb)

shapes after data prep
(99275, 497)
(42547, 497)
(60782, 497)


In [ ]:
score_train

In [ ]:
y_train_prob_pred

In [ ]:
y_test_prob_pred

### LSTM

In [ ]:
# LSTM for sequence classification in the IMDB dataset
import numpy
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
# (X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

X_train_clean_scaled, X_test_clean_scaled, y_train

# truncate and pad input sequences
max_review_length = 500
X_train_clean_scaled_seq = sequence.pad_sequences(X_train_clean_scaled, maxlen=max_review_length)
X_test_clean_scaled_seq = sequence.pad_sequences(X_test_clean_scaled, maxlen=max_review_length)

# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

### Submission Format

The classifier function wrote should return a 4d nparray with 4 columns. The columns are corresponding to the class labels: 0, 1, 2, 3. Please see examples below.

In [12]:
y_train_prob_pred

array([[9.86370744e-001, 1.36292563e-002, 3.67070949e-112,
        1.04535050e-297],
       [9.99970333e-001, 2.96674344e-005, 6.83821800e-116,
        2.37658593e-301],
       [9.99992988e-001, 7.01190145e-006, 7.24572578e-116,
        3.13840206e-300],
       ...,
       [9.93675148e-001, 7.77386317e-006, 8.49554029e-004,
        5.46752380e-003],
       [9.99998107e-001, 1.02405728e-008, 1.75735061e-007,
        1.70675826e-006],
       [9.99999309e-001, 2.68630114e-008, 8.75407480e-008,
        5.76837316e-007]])

In [13]:
y_test_prob_pred

array([[9.99999916e-01, 7.07184530e-09, 7.25436136e-09, 6.99499622e-08],
       [9.99999778e-01, 7.49671754e-09, 2.93861125e-08, 1.85256170e-07],
       [9.99999635e-01, 3.13411390e-07, 4.64843089e-08, 5.43852499e-09],
       ...,
       [9.99877224e-01, 2.50474031e-07, 2.31039598e-05, 9.94210939e-05],
       [9.99920933e-01, 2.67565958e-08, 1.78714121e-05, 6.11689183e-05],
       [9.99962674e-01, 1.17168750e-07, 1.26322837e-05, 2.45767864e-05]])

### Write test results to csv files

Please rename your file to indicate which data set you are working with. 

- If you are using the small dataset: *y_train_prob_pred_small.csv* and *y_test_prob_pred_small.csv*
- If you are using the small dataset: *y_train_prob_pred_big.csv* and *y_test_prob_pred_big.csv*
- If you are using the original dataset: *y_train_prob_pred.csv* and *y_test_prob_pred.csv*

In [14]:
pd.DataFrame(y_train_prob_pred).to_csv("y_train_prob_pred.csv")
pd.DataFrame(y_test_prob_pred).to_csv("y_test_prob_pred.csv")